In [1]:
from PIL import Image
import numpy as np

In [10]:
def crop_image(path, top, left, w, h, size=(25, 25), show=False, resize=True):
    with Image.open(path) as im:
        im = im.crop((top,left, top + h, left + w))
        if resize:
            im = im.resize(size)
        if show:
            im.show()
        return im

def to_binary(im):
    img = np.array(im.convert('L'))
    img = np.where(img>10, False, True)
    return img
    # return ~np.array(im.convert('1'))

def to_rust(im, var_name="DATA"):
    rust_def = "#[rustfmt::skip]\n"
    rust_def += f"pub const {var_name}: &[u8] = &["
    for r in im:
        rust_def += "\n"
        result = []
        buffer = "0b"
        for elem in r:
            buffer += "1" if elem else "0"
            if len(buffer) == 10:
                result.append(buffer)
                buffer = "0b"
        if len(buffer) > 2:
            # make sure buffer has at least 8 elements
            if len(buffer) < 10:
                buffer += "0" * (10-len(buffer))
            result.append(buffer)    
        rust_def += "\t" + ', '.join(result)
        rust_def += ","
    rust_def += "\n];"
    return rust_def
    

In [20]:
weather_data_rs = "// Written by ./tools/icon_processing.ipynb\n\n"

size = (32, 32)

# WEATHER_SUNNY
im = crop_image("weather.png", 25, 25, 100, 100, size=size, show=False, resize=True)
weather_data_rs += to_rust(to_binary(im), var_name="WEATHER_SUNNY")
weather_data_rs += "\n"

im = crop_image("weather.png", 175, 25, 100, 100, size=size, show=False, resize=True)
weather_data_rs += to_rust(to_binary(im), var_name="MOSTLY_CLEAR")
weather_data_rs += "\n"

im = crop_image("weather.png", 325, 25, 100, 100, show=False, resize=True)
weather_data_rs += to_rust(to_binary(im), var_name="PARTLY_CLOUDY")
weather_data_rs += "\n"

im = crop_image("weather.png", 475, 25, 100, 100, show=False, resize=True)
weather_data_rs += to_rust(to_binary(im), var_name="MOSTLY_CLOUDY")
weather_data_rs += "\n"

im = crop_image("weather.png", 25, 190, 100, 100, show=False, resize=True)
weather_data_rs += to_rust(to_binary(im), var_name="SCATTERED_SHOWERS")
weather_data_rs += "\n"

im = crop_image("weather.png", 175, 190, 100, 100, show=False, resize=True)
weather_data_rs += to_rust(to_binary(im), var_name="SHOWERS")
weather_data_rs += "\n"

im = crop_image("weather.png", 175, 520, 100, 100, show=False, resize=True)
weather_data_rs += to_rust(to_binary(im), var_name="MIST")
weather_data_rs += "\n"

im = crop_image("weather.png", 175, 355, 100, 100, show=False, resize=True)
weather_data_rs += to_rust(to_binary(im), var_name="RAIN")
weather_data_rs += "\n"

im = crop_image("wind.png", 0, 0, 32, 32, show=False, resize=False)
weather_data_rs += to_rust(~to_binary(im), var_name="WIND")
weather_data_rs += "\n"

im = crop_image("wave.png", 0, 0, 288, 392, show=False, resize=True)
weather_data_rs += to_rust(to_binary(im), var_name="WAVE")
weather_data_rs += "\n"

with open("../src/image_data.rs", "w") as f:
    f.write(weather_data_rs)